task name: Resolve StatefulCustomMapping Task
task description: Implement the StatefulCustomMapping example using a dotnet interactive notebook.

In [ ]:
#r "nuget:Microsoft.ML,1.5.5"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;

In [ ]:
#r "nuget:Microsoft.ML,1.5.5"

using System;
using System.Collections.Generic;
using Microsoft.ML;

var mlContext = new MLContext();
var samples = new List<InputData>
{
    new InputData { Sign = true },
    new InputData { Sign = true },
    new InputData { Sign = false },
    new InputData { Sign = true },
    new InputData { Sign = false },
    new InputData { Sign = true },
    new InputData { Sign = false },
    new InputData { Sign = false },
    new InputData { Sign = false },
};
var data = mlContext.Data.LoadFromEnumerable(samples);

class InputData
{
    public bool Sign { get; set; }
}

In [ ]:
class CustomMappingOutput
{
    public long SignedFibonacci { get; set; }
}

class State
{
    public long Prev { get; set; }
    public long Prev2 { get; set; }
}

Action<InputData, CustomMappingOutput, State> mapping =
    (input, output, state) =>
    {
        var fib = state.Prev2 + state.Prev;
        output.SignedFibonacci = input.Sign ? fib : -fib;
        state.Prev2 = state.Prev;
        state.Prev = fib;
    };

Action<State> init =
    state =>
    {
        state.Prev2 = 0;
        state.Prev = 1;
    };

In [ ]:
var pipeline = mlContext.Transforms.StatefulCustomMapping(mapping, init, contractName: null);
var transformer = pipeline.Fit(data);
var transformedData = transformer.Transform(data);

In [ ]:
using System;

var dataEnumerable = mlContext.Data.CreateEnumerable<TransformedData>(
    transformedData, reuseRowObject: true);

Console.WriteLine("Sign\t SignedFibonacci");
foreach (var row in dataEnumerable)
    Console.WriteLine($"{row.Sign}\t {row.SignedFibonacci}");

class TransformedData : InputData
{
    public long SignedFibonacci { get; set; }
}